# ✔	 - Import modules -

In [1]:
import os
import time
import numpy as np
import random
from matplotlib import pyplot as plt
from math import exp

import json

# ✔	 - Function definition -

# ✘ Corrected Input function

In [2]:
# Function returning the corrected Input
def Ic(Input=0, threshold=0):
    if Input - threshold <= 0:
        return 0
    else:
        return (Input - threshold)

# ✘ Noise Function

In [3]:
# White Gaussian Noise
def WGnoise(size=1):
    n = np.random.normal(0, 1, size=size)    #<==== en fonction du sigma!!!!!!!!!!!!!!!!!!!
    return n 

# ✘ Activity at time t-Delta (delay) function

In [4]:
# function to get the index "-delay/dt" in the list representing m(t-Delta)
def mdelta(synapse_value, delay, t, dt,i):
    if t < delay:  # if the delay is the time is too small, we take m(t-delta)=0
        return 0
    else:  # if the time is above or equal to the delay, we can find the m(t-delta)
        return synapse_value[int(-delay / dt)][i]

# ✘ Plotting functions (Inputs & activities) and save data function

In [5]:
def plottingI(k):
    valueGpe=[0]*n_period
    valueFsi=[0]*n_period
    valueMsn=[0]*n_period
    for stc in structures:
        for t in range(1,n_period):
        
            valueGpe[t]=I['Gpe'][t][k]
            valueFsi[t]=I['Fsi'][t][k]
            valueMsn[t]=I['Msn'][t][k]
        
    fig = plt.figure(figsize=(9,5))
    ax=fig.add_subplot(111)
    ax.plot(time_value, valueGpe, label='Msn')
    ax.plot(time_value, valueFsi, label='Gpe')
    ax.plot(time_value, valueMsn, label='Fsi')
    ax.set_title('Input over time: NEURON'+ str(k+1))
    ax.set_ylabel('Input')
    ax.set_xlabel('time')
    ax.grid()
    ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)

In [6]:
def plottingA(k):
    valueGpeMsn=[0]*n_period
    valueFsiGpe=[0]*n_period
    valueMsnFsi=[0]*n_period
    for con in connections:
        for t in range(1,n_period):
        
            valueGpeMsn[t]=m['GpeMsn'][t][k]
            valueFsiGpe[t]=m['FsiGpe'][t][k]
            valueMsnFsi[t]=m['MsnFsi'][t][k]
        
    fig = plt.figure(figsize=(9,5))
    ax=fig.add_subplot(111)
    ax.plot(time_value, valueGpeMsn, label='Msn')
    ax.plot(time_value, valueFsiGpe, label='Gpe')
    ax.plot(time_value, valueMsnFsi, label='Fsi')
    ax.set_title('Activity over time: NEURON'+ str(k+1))
    ax.set_ylabel('Activity')
    ax.set_xlabel('time')
    ax.grid()
    ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0) 

In [7]:
def data_dir(dir="os.getcwd()"):
    directory = os.path.dirname(dir)
    if not os.path.exists(os.getcwd()+str('/Data')):
        os.makedirs(os.getcwd()+str('/Data'))

In [8]:
def trial_dir(dir="os.getcwd()/Data"):
    directory = os.path.dirname(dir)
    if not os.path.exists('Data/'+str(timestr)):
        os.makedirs('Data/'+str(timestr))

# ✔	 - Initial parameters - 

In [9]:
# Population size
n = 1000

# Default Time beginning (second)
t = 0

# Default trial duration (second)
duration = 1.00   

# Default Time resolution (second)
dt = 0.0001  # 0.0001

# Period occuring 
n_period = int(duration // dt) + 1

# Initialization of the random generator (reproductibility !)
np.random.seed(1)

In [10]:
time_value = np.arange(0, duration, dt) # creat the timescale, depending on the duration and step dt, for plotting

In [11]:
connections = ["GpeMsn", "FsiGpe", "MsnFsi"] #"MsnMsn", "FsiFsi", "GpeGpe"]
structures = ["Msn", "Fsi", "Gpe"]

In [12]:
 # need IN VIVO Values + calculate them depending on the weight
h = {  
    "Msn": -51.90,   # Mahon et al., 2006. RAT (In Vitro)
    "Fsi": -47.30,   # Koos & Teper, 1999. RAT (In Vitro)
    "Gpe": -55.10,   # Abdi, Mallet et al., 2015. RAT (In Vitro)

}

 # Weights (connextions pre->post ; notation: PostPre)
G = {
    "GpeMsn": 0.50,
    "FsiGpe": 0.50,
    "MsnFsi": 0.50,
    #"MsnMsn": 1.00,
    #"FsiFsi": 1.00,
    #"GpeGpe": 1.00
}

 # tau:
tau = {
    "GpeMsn": 0.0200,
    "FsiGpe": 0.0200,
    "MsnFsi": 0.0200, #0.0150 Koos & Tepper, 1999 : (in vitro - rat)
    # "MsnMsn": 1.00,
    #"FsiFsi": 1.00,
   # "GpeGpe": 1.00
}

 # Delays:
D = {
    "GpeMsn": 0.0070, # Jaeger & Kita, 2011 ; (in vivo-rat & monkey)
    "FsiGpe": 0.0010, # Jaeger & Kita, 2011 ; (in vivo-rat & monkey)
    "MsnFsi": 0.0057, # Koos & Tepper, 1999 : (in vitro - rat)
    #"MsnMsn": 1.00,
    #"FsiFsi": 1.00,
   # "GpeGpe": 1.00  # Jaeger & Kita, 2011 ; (in vivo-rat & monkey)
}


 # Noise level (sigma)
N = {
    "Msn": 0.00,   
    "Fsi": 0.00,   
    "Gpe": 0.00
}

In [13]:
# creation of array of n neurons to store m, dm and connectivity probability(0 or 1)

m, dm, J = dict(), dict(), dict()

for con in connections:   # created an array of 'nperiod' array(to store the value trought the time) with n zeroes
    m[con] = [[0 for x in range(n)] for y in range(n_period)]
    dm[con] = [[0 for x in range(n)] for y in range(n_period)]
    
    
for con in connections:
    if con  == 'GpeMsn':
        Co=1.000
    elif con == 'MsnFsi':
        Co=1.000
    elif con == 'FsiGpe':
        Co=1.000
    J[con] = np.reshape([[np.random.choice(2,n, p=[(1-Co),Co])] for y in range(n)],(n,n))
    #print([con],J[con])

In [14]:
# inputs
I = dict()
for stc in structures: # create an array of 'nperiod' array(to store the value trought the time) with n zeroes
    I[stc] = [[0 for x in range(n)] for y in range(n_period)]

In [ ]:
input_mapping = {
    "Msn": "MsnFsi",
    "Gpe": "GpeMsn",
    "Fsi": "FsiGpe"
}

activities_mapping = {
    "GpeMsn": "Msn",
    "FsiGpe": "Gpe",
    "MsnFsi": "Fsi"

}

# ✔	 - Simulation Core -

In [ ]:
for t in range(1, n_period):
    
    for stc in structures:
        for i in range(n):
            if stc == "Msn" or "Gpe" or "Fsi":
                con = input_mapping[stc]
                I[stc][t][i] = - sum([G[con] * x * (mdelta(m[con][:t + 1], D[con], time_value[t], dt,i)) for x in J[con][i]]) + WGnoise()
            else:
                pass



    for con in connections:
        for i in range(n):
            # Update of activities
            stc = activities_mapping[con]
            dm[con][t][i] = dt * (- m[con][t][i] + Ic(I[stc][t][i], h[stc])) / tau[con] #+ float(noise(1, N[stc]))
        if t < n_period-1:
            m[con][t+1]=[x+y for x, y in zip(m[con][t],dm[con][t])]
        else:
            pass

# ✔	 - Result Plots & Save data -

In [ ]:
timestr = time.strftime("%Y-%m-%d-%H:%M:%S")
data_dir()
trial_dir()

np.save('Data/'+str(timestr)+str('/Msn_Inputs'),I["Msn"])     # here saved in npy format(save in txt format?)
np.save('Data/'+str(timestr)+str('/Gpe_Inputs'),I["Gpe"])
np.save('Data/'+str(timestr)+str('/Fsi_Inputs'),I["Fsi"])

np.save('Data/'+str(timestr)+str('/GpeMsn_Activities'),m["GpeMsn"])
np.save('Data/'+str(timestr)+str('/MsnFsi_Activities'),m["MsnFsi"])
np.save('Data/'+str(timestr)+str('/FsiGpe_Activities'),m["FsiGpe"])



# ===> creat file with all the DATA USED!!( weight etc) JSON?


datax={'Weight':G,
       'Threshold':h,
       'Delay':D,
       'tau':tau,
       'Noise':N,
       'Neuron_number':n,
       'time_trial':t,
       'duration':duration,
       'step':dt}

with open('Data/'+str(timestr)+'/0-Parameters.json','w') as f:
        for i in datax:
            json.dump(datax[i], f)
            f.write('\n')
            

In [ ]:
# Plotting INPUTS
for k in range(0,n):
    plottingI(k)
    plt.savefig('Data/'+str(timestr)+str('/In'+str(k+1)+'.png'),dpi=300,bbox_inches='tight')
    plt.show()

In [ ]:
# Plotting ACTIVITIES
for k in range(0,n):
    plottingA(k)
    plt.savefig('Data/'+str(timestr)+str('/mn'+str(k+1)+'.png'),dpi=300,bbox_inches='tight')
    plt.show()

# ✔	 - Mean firing rate plotting - 

In [ ]:
MeaniGM=list()
MeaniMF=list()
MeaniFG=list()
for con in connections:
    for t in range(0,n_period):
        if con  == 'GpeMsn':
            MeaniGM.append(np.mean(m['GpeMsn'][t]))
        elif con == 'MsnFsi':
            MeaniMF.append(np.mean(m['MsnFsi'][t]))
        elif con == 'FsiGpe':
            MeaniFG.append(np.mean(m['FsiGpe'][t]))  

plt.plot(time_value,MeaniGM,label='Msn')
plt.plot(time_value,MeaniFG,label='Gpe')
plt.plot(time_value,MeaniMF,label='Fsi')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.grid()
plt.show()